# Automated Program Repair

Automated Program Repair (APR) aims to fix faults in programs with as little human intervention as possible. To automatically generate patches (candidates to fix our programs) we normally base our efforts on the original buggy program. This approach takes into account that developers implement programs that are *almost* accurate.

# Large Language Models

Large language models (LLMs) are AI systems that are pre-trained on vast amounts of textual data, such as books, articles, websites and also code. These models use deep learning algorithms to analyze patterns and structures in both natural and programming languages. That way, they produce human-like text or code, generating coherent and relevant responses across various tasks.
Some example of models are:

* **Natural language:** BERT, GPT, etc.
* **Programming language:** CodeBERT, CodeGPT, Codex
* **Chat assistants:** ChatGPT, YOU, HuggingChat
* **LLMs:** GPT-3, GPT-4, OPT-175B, BLOOM-176B.

  Try OPT-175B online: https://opt.alpa.ai/

# Setup

In [ ]:
!pip install transformers
!pip install openai
!git clone https://gitlab.com/FranciscoRibeiro/apr-llm-tutorial -b notebook
!mv apr-llm-tutorial/* .
!rm -r apr-llm-tutorial

# Mask Filling

*Mask filling* is a task in which, during training, some tokens are hidden (masked) from input sequences and the model learns to predict the original values based on the context of the surrounding tokens.

Before moving on, let's consider the following Java class for a college degree containing the enrolled students:



```java
public class Degree {
    private List<Student> students;
    ...
    public boolean containsStudent(String id) {
        return students.stream()
                       .filter(x -> x.getId().equals(id))
                       .count() < 0; // wrong operator
    }
```

The method `containsStudent` is incorrect. The comparison in line 7 should be if the number of students is **greater than zero** (`> 0`).

We could address this issue by exploring different code alternatives for the place we suspect is wrong. Mask filling can help us with that.

`codebert-base-mlm` is a pre-trained model we can use for the masked modeling objective.
The *transformers* library makes this process very easy:

In [ ]:
from transformers import pipeline

code = ".count() <mask> 0;" # Buggy line 7: notice we mask the operator

# Simple way to ask CodeBERT to fill in the mask
fill_mask = pipeline('fill-mask', model="microsoft/codebert-base-mlm")
outputs = fill_mask(code)

# Check the generations
print("\nGenerations:")
for o in outputs:
    print(o)

The output will look something like this:
```javascript
{'score': 0.7650061845779419, 'token': 5457, 'token_str': ' ='}, ...}
{'score': 0.10670557618141174, 'token': 45994, 'token_str': ' =='}, ...}
{'score': 0.059551902115345, 'token': 671, 'token_str': ' return'}, ...}
{'score': 0.024524779990315437, 'token': 8061, 'token_str': ' >'}, ...}
{'score': 0.008818789385259151, 'token': 49333, 'token_str': '!='}, ...}    
```

-   `token_str` represents the string value that should replace
    `<mask>`, i.e. it is the predicted token;

-   `score` is the likelihood the model associates to the predicted
    token, i.e. the confidence that this is the correct token;

-   `token` is the unique ID for the predicted token. Internally, the
    model represents tokens in this form in a structure called the
    *vocabulary*.

The majority of suggestions do not help us that much. Although the
operator `>` is ranked 4th, we can still do better. Mask filling works
by analyzing the surrounding context, so we could try and provide more
information so that the model can make better predictions.


### Exercise 1:
Modify the *input sequence* in `code` so that predictions are more
adequate.

In [ ]:
from transformers import pipeline

# Edit the 'code' variable
code = "..."

# Simple way to ask CodeBERT to fill in the mask
fill_mask = pipeline('fill-mask', model="microsoft/codebert-base-mlm")
outputs = fill_mask(code)

# Check the generations
print("\nGenerations:")
for o in outputs:
    print(o)

<details>
  <summary><font size="3" color="white"><b>Click to show/hide hints</b></font></summary>


Provide the whole statement:
```python
code = "return students.stream().filter(x -> x.getId().equals(id)).count() <mask> 0;"
```

Provide the whole method:
```python
code = """public boolean containsStudent(String id) {
        return students.stream().filter(x -> x.getId().equals(id)).count() <mask> 0;
    }"""
```

### Exercise 2:
*CodeBERT* was trained on a dataset containing Python, Javascript, Ruby,
Go, Java, and PHP. We can also use the *transformers* library to predict
multiple masks. Consider the Python function:

In [ ]:
# Computes intersection of two lists
def find_common(list1, list2):
    common = []
    for elem in list1:
        if elem in list2 and elem not in common:
            list1.add(elem) # buggy line: 'list1.add' is incorrect
    return common

Use the above function as an *input sequence* and place 3 `<mask>`'s
to replace the expression `list1.add`.

In [ ]:
from transformers import pipeline

# Edit the 'code' variable
code = """..."""

# Simple way to ask CodeBERT to fill in the mask
fill_mask = pipeline('fill-mask', model="microsoft/codebert-base-mlm")
outputs = fill_mask(code)

# Check the generations
print("\nGenerations:")
for o in outputs:
    print(o)

Fix the `find_common` function according to the predicted tokens and run the respective cell.

After that, run the next cell to test the function.

In [ ]:
from utils import *
test_find_common(find_common)

# Open-ended Generation

The process of open-ended generation involves using a language model to generate a sequence of tokens based on the provided input. This input context includes code that precedes the point from which the model is asked to start generating additional code. The generated code sequence essentially continues the original code piece and should be coherent and relevant to the context provided to the model. *CodeGPT* is a model based on GPT-2 that performs this task. Let's look at a real-world bug from a Java program this time:

```java
505c505
... .toArray(new String[m.size()])); // bug
---
... .toArray(new String[q.getValue().size()])); // fix
```

The previous *diff* output highlights the changes that line 505 needs in order to be fixed. However, in a real scenario, we don't have access to a program's correct version. Can we use the buggy code to generate the necessary patch? Let's consider the code up until the point where we want *CodeGPT* to start completing it. Discarded code is to the right of the cursor (`|`) character.

```java
... .toArray(new String[|m.size()]));
```


File `prompts/atmosphere_65ce3...3437.java` reflects this.

Run the following code block:

In [ ]:
from generate import *

setup()

# filename to analyze
filename = "prompts/atmosphere_65ce31321f736b92ef7b5a9f8890c314e1313437.java"
tokenizer, model = load() # Load the tokenizer and the model separately

# Utility function to generate completions for the input file
tokens_used, completions = complete_file(filename, tokenizer, model)

print(tokens_used) # Show used context (default: previous 1000 tokens)
print("\nGenerations:") # Show generations
for c in completions:
    print(c)
  

The output will look like this:
```
Generations:
0]));} return newM; }/*** Set the request parameters.** @param request
```

Considering the output above, *CodeGPT* generates index `0` (zero) and some more code. It seems it's not helping us out-of-the-box. However, we can adjust the generation process by passing more parameters to the `complete_file` function.

### Exercise 3
Parameterize the code block below to output multiple generations (`n_generations` parameter).

### Exercise 4
Introduce variability throughout the generations (`variability` parameter can be `True` or `False`).

### Exercise 5
Stop generations at a specific point (`stop_at` can be set to a specific character).

### Exercise 6 (*Optional*)
Experiment with the other examples of real-world bugs in `prompts/`.

In [ ]:
from generate import *

setup()

# filename to analyze
filename = "prompts/atmosphere_65ce31321f736b92ef7b5a9f8890c314e1313437.java"
tokenizer, model = load() # Load the tokenizer and the model separately

# Utility function to generate completions for the input file
tokens_used, completions = complete_file(filename, tokenizer, model
                                         # , n_generations= # int
                                         # , variability= # bool
                                         # , stop_at= # char
                            )
print(tokens_used) # Show used context (default: previous 1000 tokens)
print("\nGenerations:") # Show generations
for c in completions:
    print(c)
  

# Prompt Engineering

Prompt engineering refers to the process of designing the input text (prompt) in a way that conveys or guides the model into generating the desired response. This makes LLM's more flexible by enabling them to generate responses to a wide variety of tasks, as we have a way to have models produce coherent output to virtually any question. It's almost like mimicking artificial general intelligence (AGI). Consider the following Python function to calculate the first `n` prime numbers:

In [ ]:
def calculate_prime_numbers(n):
    prime_numbers = []
    i = 2
    while len(prime_numbers) <= n:
        is_prime = False
        for j in range(2, i):
            if i % j == 0:
                is_prime = True
                break
        if is_prime:
            prime_numbers.add(i)
        i += 1
    return prime_numbers

The implementation is faulty. Can you spot the mistake(s)? If you want, take 2 minutes to fix it by yourself. Either way, let's see how we could use prompt engineering to have GPT-3 generate a correct version. Suppose the following template for a prompt. Instructions surrounded with `<...>` mean comment syntax should be used:
```
<programming language>
<suggest the code is buggy>
... buggy code ...

<say we want the fixed version>
```

### Exercise 7
Based on the previous template, build the corresponding prompt for
`calculate_prime_numbers`. (Don't delete `%%writefile ...`)

<details>
  <summary><font size="3" color="white"><b>Click to show/hide hints</b></font></summary>

The prompt should look something like this:
```python
# Python
# Buggy
def calculate_prime_numbers(n):
    prime_numbers = []
    i = 2
    while len(prime_numbers) <= n:
        is_prime = False
        for j in range(2, i):
            if i % j == 0:
                is_prime = True
                break
        if is_prime:
            prime_numbers.add(i)
        i += 1
    return prime_numbers

# Fixed
```

In [ ]:
%%writefile primes_prompt.txt
YOUR PROMPT HERE

### Exercise 8
After saving the prompt you built above (by running the respective code block), place your OpenAI API key below and run the next code block:

In [ ]:
%%writefile openai.api_key
YOUR API KEY HERE

In [ ]:
from gpt import *

setup()
prompt_file = "primes_prompt.txt"

response = mk_request(prompt_file, mode="complete")
show_response(response)

In [ ]:
# Don't forget to edit and run the previous code
# with calculate_prime_number's definition
calculate_prime_numbers(10)

Suppose we are now aware of the test cases the function should satisfy:
```
calculate_prime_numbers(5) == [2, 3, 5]
calculate_prime_numbers(18) == [2, 3, 5, 7, 11, 13, 17]
calculate_prime_numbers(23) == [2, 3, 5, 7, 11, 13, 17, 19, 23]
calculate_prime_numbers(31) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
```

According to the test cases, the function should **not** calculate the
first `n` prime numbers but the prime numbers <u>up to</u> `n`.

### Exercise 9
Through prompt engineering, create an input that makes the model output
an intended implementation of `calculate_prime_numbers`.


<details>
  <summary><font size="3" color="white"><b>Click to show/hide hints</b></font></summary>

The prompt could look something like this:
```python
# Python
# Test cases
calculate_prime_numbers(5) == [2, 3, 5]
calculate_prime_numbers(18) == [2, 3, 5, 7, 11, 13, 17]
calculate_prime_numbers(23) == [2, 3, 5, 7, 11, 13, 17, 19, 23]
calculate_prime_numbers(31) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]

# Function passing the test cases
```

Or, an alternative:
```python
#Python
#Buggy Code:
#compute the first n primes
def calculate_prime_numbers(n):
    prime_numbers = []
    i = 2
    while len(prime_numbers) <= n:
        is_prime = True
        for j in range(2, i):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            prime_numbers.append(i)
        i += 1
    return prime_numbers
#Fixed Code:
#compute primes up to number n
```

There are many ways to approach the problem. Try different ones.

In [ ]:
%%writefile primes_tests_prompt.txt
YOUR PROMPT HERE

In [ ]:
from gpt import *

setup()
prompt_file = "primes_tests_prompt.txt"

response = mk_request(prompt_file, mode="complete")
show_response(response)

In [ ]:
# Don't forget to edit and run the previous code
# with calculate_prime_number's definition
calculate_prime_numbers(10)

# Editing through Instructions

There is another way to interact with GPT-3 through instructions (called *edit* mode) that are separate from the prompt. Sometimes, this can facilitate our efforts as we can isolate the changes we want to see without messing with the input.

### Exercise 10
Consider the previous `calculate_prime_numbers` function and generate different alternatives by instructing GPT to:
- use the sieve of eratosthenes;
- use a generator function;
- use memoization.


In [ ]:
# Fixed version; but you are free to experiment with any version you want
%%writefile primes_function_fixed.txt
def calculate_prime_numbers(n):
    prime_numbers = []
    i = 2
    while len(prime_numbers) < n:
        is_prime = True
        for j in range(2, i):
            if i % j == 0:
                is_prime = False
                break
        if is_prime:
            prime_numbers.append(i)
        i += 1
    return prime_numbers

In [ ]:
from gpt import *

setup()
prompt_file = "primes_function_fixed.txt"

response = mk_request(prompt_file,
                      mode="", # change mode (complete, edit or insert)
                      instructions="") # If "edit", provide instructions
show_response(response)

In [ ]:
# Use this code block to try the generated implementations
def calculate_prime_numbers(n):
  # Some GPT-3 generated implementation


In [ ]:
calculate_prime_numbers(10)

A faulty program does not necessarily crash or output incorrect results. As we saw in the previous exercises, we may wish to change an implementation so that it is more efficient. In a way, we can consider we are still repairing the program. What if an app makes your phone spend more energy than it has to? We can look at it as being faulty.

Below, there is a method which does not take into account the battery status when performing a demanding operation over the network. This is taken from a real-world bug.

### Exercise 11
Instruct GPT-3 to prevent sync if battery is low.

In [ ]:
%%writefile energy_battery_prompt.txt
public void onPerformSync(Account account, Bundle extras, String authority, ContentProviderClient provider, SyncResult syncResult) {
		ConnectivityManager manager = (ConnectivityManager) context.getSystemService(Context.CONNECTIVITY_SERVICE);
		NetworkInfo networkInfo = manager.getActiveNetworkInfo();
		
		// Don't try to sync if no network!
		if(networkInfo == null || !networkInfo.isConnected() || Util.isOffline(context)) {
			Log.w(TAG, "Not running sync, not connected to network");
			return;
		}

		// Check if user wants to only sync on wifi
		SharedPreferences prefs = Util.getPreferences(context);
		if(prefs.getBoolean(Constants.PREFERENCES_KEY_SYNC_WIFI, true)) {
			if(networkInfo.getType() == ConnectivityManager.TYPE_WIFI) {
				executeSync(context);
			} else {
				Log.w(TAG, "Not running sync, not connected to wifi");
			}
		} else {
			executeSync(context);
		}
	}

In [ ]:
from gpt import *

setup()
prompt_file = "energy_battery_prompt.txt"

response = mk_request(prompt_file,
                      mode="", # change mode (complete, edit or insert)
                      instructions="") # If "edit", provide instructions
show_response(response)

# Filling in arbitrary tokens

*Insert* mode in GPT-3 can be used to recreate the effects of mask filling
with an arbitrary number of tokens by allowing the user to insert tokens
into partially complete code. We are not limited to a single predicted
token and we can insert multiple tokens while still using only one
mask.

### Exercise 12
Use any of the example programs you saw throughout this tutorial and
build prompts to add code in the middle of content. Note that GPT-3
expects an `[insert]` tag instead of a `<mask>` tag.

In [ ]:
# Build the prompt you like and make sure you have only one "[insert]" tag
# Below is just an example, edit it as you like
%%writefile exercise_12_prompt.txt
def foo(a, b):
  return [insert]

# Should be true
foo(1,2) == 3
foo(5,5) == 10

In [ ]:
from gpt import *

setup()
prompt_file = "exercise_12_prompt.txt"

response = mk_request(prompt_file,
                      mode="insert") # change mode (complete, edit or insert)
show_response(response)